In [1]:
# @title Main Launcher Cell - Section 1: Auto-Clone & Setup  { display-mode: "form" }
# @markdown ▶️ **Click RUN** – Auto-clones your repo and sets up basic imports

# ───────────────────────────────────────────────────────────────
# AUTO-CLONE / UPDATE YOUR REPO (must execute before imports)
# ───────────────────────────────────────────────────────────────
import os, sys, subprocess, pathlib
REPO_URL  = "https://github.com/remphanostar/WebUnified.git"
CLONE_DIR = "/content/WebUnified"

def _sh(cmd:str):
    print(f"$ {cmd}")
    p = subprocess.Popen(cmd, shell=True, text=True)
    p.communicate()
    if p.returncode:
        raise RuntimeError(f"Command failed ({p.returncode})")

if not os.path.isdir(CLONE_DIR):
    print("📥 cloning WebUnified …")
    _sh(f"git clone --depth 1 {REPO_URL} {CLONE_DIR}")
else:
    print("🔄 pulling latest …")
    _sh(f"git -C {CLONE_DIR} pull --ff-only")

if CLONE_DIR not in sys.path:
    sys.path.append(CLONE_DIR)
    print("🧩 repo added to sys.path")

REQ = pathlib.Path(CLONE_DIR, "requirements-launcher.txt")
if REQ.exists():
    print("📦 installing launcher deps (cached if previously run) …")
    _sh(f"pip install -q -r {REQ}")

print("✅ repo ready at", CLONE_DIR)

# ───────────────────────────────────────────────────────────────
# BASIC IMPORTS
# ───────────────────────────────────────────────────────────────
import json
import time
import threading
import queue
import psutil
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Optional, Any

# Gradio and UI imports
import gradio as gr
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

print("✅ Basic imports successful")


🔄 pulling latest …
$ git -C /content/WebUnified pull --ff-only
🧩 repo added to sys.path
📦 installing launcher deps (cached if previously run) …
$ pip install -q -r /content/WebUnified/requirements-launcher.txt
✅ repo ready at /content/WebUnified
✅ Basic imports successful


In [2]:
# @title Main Launcher Cell - Section 2: Configuration Setup  { display-mode: "form" }
# @markdown ▶️ **Creates complete config with model_categories section**

# ───────────────────────────────────────────────────────────────
# CONFIGURATION SETUP WITH MODEL_CATEGORIES
# ───────────────────────────────────────────────────────────────

CONFIG_FILE = "config.json"
WORKSPACE_DIR = "/workspace"
MODELS_DIR = "/data/models"

# Create default config if it doesn't exist
if not os.path.exists(CONFIG_FILE):
    print("⚠️ Config file not found. Creating default configuration...")

    # Complete default configuration with model_categories
    default_config = {
        "workspace_settings": {
            "workspace_dir": WORKSPACE_DIR,
            "models_dir": MODELS_DIR,
            "log_level": "INFO",
            "auto_create_dirs": True,
            "max_log_lines": 1000
        },
        "tools": {
            "automatic1111": {
                "name": "AUTOMATIC1111",
                "dir": "automatic1111",
                "repo": "https://github.com/AUTOMATIC1111/stable-diffusion-webui.git",
                "script": "launch.py",
                "python_version": "3.10.6",
                "venv_name": "a1111_venv",
                "install_cmd": "{pip} install -r requirements_versions.txt",
                "default_args": ["--share", "--xformers", "--no-half-vae", "--medvram"],
                "centralization_method": "cli_args",
                "centralization_args": [
                    "--ckpt-dir", "{models_dir}/Stable-diffusion",
                    "--vae-dir", "{models_dir}/VAE",
                    "--lora-dir", "{models_dir}/Lora",
                    "--embeddings-dir", "{models_dir}/embeddings"
                ],
                "description": "The foundational WebUI for Stable Diffusion",
                "port": 7862,
                "status": "stopped",
                "risk_level": "low",
                "icon": "🎨",
                "category": "stable_diffusion"
            },
            "sdnext": {
                "name": "SD.Next",
                "dir": "sdnext",
                "repo": "https://github.com/vladmandic/automatic.git",
                "script": "launch.py",
                "python_version": "3.11",
                "venv_name": "sdnext_venv",
                "install_cmd": "{pip} install -r requirements.txt",
                "default_args": ["--use-xformers", "--backend", "diffusers", "--medvram", "--share"],
                "centralization_method": "cli_args",
                "centralization_args": ["--models-dir", "{models_dir}"],
                "description": "Next-generation interface with cutting-edge features",
                "port": 7860,
                "status": "stopped",
                "risk_level": "low",
                "icon": "🚀",
                "category": "stable_diffusion"
            }
        },
        "hardware_profiles": {
            "high_vram": {
                "name": "High VRAM (16GB+)",
                "description": "For RTX 3090/4090, A100, etc.",
                "args": ["--always-high-vram", "--no-lowvram"],
                "icon": "🚀"
            },
            "medium_vram": {
                "name": "Medium VRAM (8-16GB)",
                "description": "For RTX 3070/4070, RTX 3080, etc.",
                "args": ["--medvram"],
                "icon": "⚡"
            },
            "low_vram": {
                "name": "Low VRAM (<8GB)",
                "description": "For RTX 3060, GTX 1080, etc.",
                "args": ["--lowvram", "--opt-channelslast"],
                "icon": "💾"
            }
        },
        # THIS IS THE MISSING SECTION THAT WAS CAUSING THE ERROR:
        "model_categories": {
            "Stable-diffusion": {
                "description": "Checkpoint models (.safetensors, .ckpt)",
                "extensions": [".safetensors", ".ckpt"],
                "size_estimate_gb": 4.0
            },
            "Lora": {
                "description": "LoRA and LyCORIS models",
                "extensions": [".safetensors", ".pt"],
                "size_estimate_gb": 0.1
            },
            "VAE": {
                "description": "VAE models (.pt, .safetensors)",
                "extensions": [".safetensors", ".pt"],
                "size_estimate_gb": 0.8
            },
            "ControlNet": {
                "description": "ControlNet models",
                "extensions": [".safetensors", ".pth"],
                "size_estimate_gb": 1.5
            },
            "embeddings": {
                "description": "Textual Inversion embeddings",
                "extensions": [".pt", ".safetensors"],
                "size_estimate_gb": 0.01
            },
            "ESRGAN": {
                "description": "Upscaler models",
                "extensions": [".pth"],
                "size_estimate_gb": 0.2
            }
        }
    }

    with open(CONFIG_FILE, 'w') as f:
        json.dump(default_config, f, indent=2)

    print("✅ Default configuration created with model_categories")

print("✅ Configuration setup complete")


✅ Configuration setup complete


In [3]:
# @title Main Launcher Cell - Section 3: Manager Init & CSS  { display-mode: "form" }
# @markdown ▶️ **Initializes the manager and loads CSS styling**

# ───────────────────────────────────────────────────────────────
# MANAGER INITIALIZATION
# ───────────────────────────────────────────────────────────────

# Import our enhanced manager
try:
    from manage_venvs import EnhancedMultiVenvManager
except ImportError:
    print("❌ manage_venvs.py not found. Please ensure all files are in the same directory.")
    raise

# Initialize the enhanced manager
try:
    manager = EnhancedMultiVenvManager(CONFIG_FILE)
    print(f"✅ Manager initialized")
    print(f"📁 Workspace: {manager.workspace_dir}")
    print(f"🎨 Models: {manager.models_dir}")
    print(f"🛠️ Tools: {len(manager.config['tools'])}")
except Exception as e:
    print(f"❌ Failed to initialize manager: {e}")
    raise

# ───────────────────────────────────────────────────────────────
# ADVANCED CSS STYLING
# ───────────────────────────────────────────────────────────────

advanced_css = """
:root {
    --primary-color: #4a9eff;
    --success-color: #10b981;
    --warning-color: #f59e0b;
    --error-color: #ef4444;
    --bg-primary: #0f172a;
    --bg-secondary: #1e293b;
    --bg-tertiary: #334155;
    --text-primary: #f1f5f9;
    --text-secondary: #94a3b8;
    --border-color: #475569;
}

.gradio-container {
    background: var(--bg-primary) !important;
    color: var(--text-primary) !important;
    font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
}

.tool-card {
    background: linear-gradient(135deg, var(--bg-secondary), var(--bg-tertiary));
    border: 1px solid var(--border-color);
    border-radius: 12px;
    padding: 1.5rem;
    margin-bottom: 1rem;
    box-shadow: 0 4px 6px -1px rgba(0, 0, 0, 0.1);
    transition: all 0.3s ease;
    position: relative;
    overflow: hidden;
}

.tool-card::before {
    content: '';
    position: absolute;
    top: 0;
    left: 0;
    width: 100%;
    height: 3px;
    background: linear-gradient(90deg, var(--primary-color), #764ba2);
    transform: translateX(-100%);
    transition: transform 0.3s ease;
}

.tool-card:hover::before {
    transform: translateX(0);
}

.tool-card:hover {
    transform: translateY(-2px);
    box-shadow: 0 10px 16px -4px rgba(0, 0, 0, 0.1);
    border-color: var(--primary-color);
}

.status-dot {
    height: 12px;
    width: 12px;
    border-radius: 50%;
    display: inline-block;
    margin-right: 8px;
    transition: all 0.3s ease;
}

.status-running {
    background-color: var(--success-color);
    box-shadow: 0 0 8px var(--success-color);
    animation: pulse-success 2s infinite;
}

.status-stopped {
    background-color: var(--error-color);
    box-shadow: 0 0 4px var(--error-color);
}

.status-starting {
    background-color: var(--warning-color);
    box-shadow: 0 0 6px var(--warning-color);
    animation: pulse-warning 1.5s infinite;
}

@keyframes pulse-success {
    0% { box-shadow: 0 0 0 0 rgba(16, 185, 129, 0.7); }
    70% { box-shadow: 0 0 0 10px rgba(16, 185, 129, 0); }
    100% { box-shadow: 0 0 0 0 rgba(16, 185, 129, 0); }
}

@keyframes pulse-warning {
    0% { box-shadow: 0 0 0 0 rgba(245, 158, 11, 0.7); }
    70% { box-shadow: 0 0 0 8px rgba(245, 158, 11, 0); }
    100% { box-shadow: 0 0 0 0 rgba(245, 158, 11, 0); }
}

.global-settings {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    color: white;
    border-radius: 12px;
    padding: 2rem;
    margin-bottom: 2rem;
    box-shadow: 0 8px 16px rgba(0, 0, 0, 0.2);
}

.metric-card {
    background: var(--bg-secondary);
    border-radius: 8px;
    padding: 1rem;
    text-align: center;
    border: 1px solid var(--border-color);
}

.metric-value {
    font-size: 2rem;
    font-weight: bold;
    color: var(--primary-color);
}

.metric-label {
    color: var(--text-secondary);
    font-size: 0.875rem;
    margin-top: 0.25rem;
}

.risk-low { border-left: 4px solid var(--success-color); }
.risk-medium { border-left: 4px solid var(--warning-color); }
.risk-high { border-left: 4px solid var(--error-color); }

.log-viewer {
    background: var(--bg-primary);
    border: 1px solid var(--border-color);
    border-radius: 6px;
    font-family: 'Monaco', 'Menlo', 'Consolas', monospace;
    max-height: 300px;
    overflow-y: auto;
    font-size: 0.875rem;
    line-height: 1.4;
}
"""

print("✅ CSS styling loaded")
print("✅ Manager and styling ready")


✅ Manager initialized
📁 Workspace: /workspace
🎨 Models: /data/models
🛠️ Tools: 2
✅ CSS styling loaded
✅ Manager and styling ready


In [4]:
# @title Main Launcher Cell - Section 4: Process Tracker  { display-mode: "form" }
# @markdown ▶️ **Creates process tracking and helper classes**

# ───────────────────────────────────────────────────────────────
# PROCESS TRACKER CLASS
# ───────────────────────────────────────────────────────────────

class ProcessTracker:
    """Enhanced process tracking with UI updates"""

    def __init__(self, manager):
        self.manager = manager
        self.status_cache = {}
        self.log_cache = {}

    def get_status_summary(self):
        """Get overall system status"""
        running_count = 0
        total_count = len(self.manager.config['tools'])

        for tool_id in self.manager.config['tools'].keys():
            status = self.manager.get_process_status(tool_id)
            if status.get('status') == 'running':
                running_count += 1

        return {
            'running': running_count,
            'total': total_count,
            'stopped': total_count - running_count
        }

    def get_tool_status_html(self, tool_id):
        """Generate HTML for tool status"""
        tool_config = self.manager.config['tools'][tool_id]
        status = self.manager.get_process_status(tool_id)
        current_status = status.get('status', 'stopped')

        status_class = f"status-{current_status}"

        html = f"""
        <div class="tool-card risk-{tool_config['risk_level']}">
            <div style="display: flex; justify-content: space-between; align-items: center;">
                <h3>{tool_config['icon']} {tool_config['name']}</h3>
                <span class="status-dot {status_class}" id="status-{tool_id}"></span>
            </div>
            <p>{tool_config['description']}</p>
            <div style="margin-top: 0.5rem; color: var(--text-secondary);">
                <span>Status: <strong>{current_status.title()}</strong></span>
                <span style="margin-left: 1rem;">Port: {tool_config['port']}</span>
                <span style="margin-left: 1rem;">Risk: {tool_config['risk_level'].title()}</span>
            </div>
        </div>
        """

        return html

    def update_logs(self, tool_id):
        """Get updated logs for a tool"""
        logs = self.manager.get_logs(tool_id, max_lines=50)
        if logs:
            return '\n'.join(logs)
        return "No logs available"

# ───────────────────────────────────────────────────────────────
# HELPER FUNCTIONS
# ───────────────────────────────────────────────────────────────

def print_launch_info():
    """Print system information before launch"""
    print("\n" + "="*60)
    print("🚀 UNIFIED WEBUI LAUNCHER STARTING")
    print("="*60)
    print(f"📁 Workspace: {manager.workspace_dir}")
    print(f"🎨 Models: {manager.models_dir}")
    print(f"🛠️ Tools: {len(manager.config['tools'])}")

    # System resources
    try:
        print(f"💾 RAM: {psutil.virtual_memory().available / (1024**3):.1f} GB available")
        print(f"💿 Disk: {psutil.disk_usage(str(manager.workspace_dir)).free / (1024**3):.1f} GB available")
    except:
        pass

    print("\n🎯 Features:")
    print("   ✅ Multi-environment isolation (zero conflicts)")
    print("   ✅ Centralized asset management (storage optimization)")
    print("   ✅ Real-time process monitoring")
    print("   ✅ Advanced Gradio interface")
    print("   ✅ Batch operations support")
    print("   ✅ Hardware profile optimization")

    print("\n📋 Next Steps:")
    print("   1. Use 'Setup All Environments' to install WebUIs")
    print("   2. Place your models in the centralized directory")
    print("   3. Launch any WebUI with optimal settings")
    print("   4. Monitor processes and logs in real-time")
    print("="*60)
    print()

# Initialize process tracker
process_tracker = ProcessTracker(manager)
print("✅ Process tracker initialized")
print("✅ Helper functions loaded")


✅ Process tracker initialized
✅ Helper functions loaded


In [5]:
# @title Main Launcher Cell - Section 5: Gradio Interface Part 1  { display-mode: "form" }
# @markdown ▶️ **Creates the main Gradio interface structure**

# ───────────────────────────────────────────────────────────────
# GRADIO INTERFACE CREATION - PART 1
# ───────────────────────────────────────────────────────────────

def create_advanced_interface():
    """Create the complete advanced Gradio interface"""

    with gr.Blocks(css=advanced_css, title="Unified WebUI Launcher", theme=gr.themes.Base()) as demo:

        # Global state management
        process_states = gr.State({})
        refresh_counter = gr.State(0)

        # Header section
        gr.HTML("""
        <div class="global-settings">
            <h1>🚀 Unified Multi-WebUI Stable Diffusion Launcher</h1>
            <p>Advanced multi-environment management with centralized assets and zero conflicts</p>
        </div>
        """)

        # System status dashboard
        with gr.Row():
            with gr.Column(scale=2):
                gr.Markdown("### 🌐 Global Configuration")

                with gr.Row():
                    workspace_dir = gr.Textbox(
                        label="Workspace Directory",
                        value=str(manager.workspace_dir),
                        interactive=False
                    )
                    models_dir = gr.Textbox(
                        label="Models Directory",
                        value=str(manager.models_dir),
                        interactive=False
                    )

                hardware_profile = gr.Dropdown(
                    label="Hardware Profile",
                    choices=[(profile['name'], key) for key, profile in manager.config['hardware_profiles'].items()],
                    value="medium_vram",
                    interactive=True
                )

            with gr.Column(scale=1):
                gr.Markdown("### 📊 System Status")

                status_summary = process_tracker.get_status_summary()

                with gr.Row():
                    total_tools_display = gr.HTML(
                        f'<div class="metric-card"><div class="metric-value">{status_summary["total"]}</div><div class="metric-label">Tools Available</div></div>'
                    )
                    running_count_display = gr.HTML(
                        f'<div class="metric-card"><div class="metric-value">{status_summary["running"]}</div><div class="metric-label">Currently Running</div></div>'
                    )

        # Tool management section
        gr.Markdown("## 🛠️ WebUI Tool Management")

        # Create tabs for each tool category
        with gr.Tabs():

            # Stable Diffusion tools tab
            with gr.TabItem("🎨 Stable Diffusion Tools"):
                sd_tools = {k: v for k, v in manager.config['tools'].items()
                           if v.get('category') == 'stable_diffusion'}

                tool_components = {}

                for tool_id, tool_config in sd_tools.items():
                    with gr.Group():
                        # Tool status display
                        tool_status_html = gr.HTML(
                            process_tracker.get_tool_status_html(tool_id),
                            elem_id=f"tool-status-{tool_id}"
                        )

                        with gr.Row():
                            with gr.Column(scale=3):
                                # Configuration options
                                custom_args = gr.Textbox(
                                    label="Custom Arguments",
                                    value=" ".join(tool_config.get('default_args', [])),
                                    placeholder="Additional launch arguments...",
                                    interactive=True
                                )

                            with gr.Column(scale=1):
                                # Control buttons
                                launch_btn = gr.Button(
                                    f"🚀 Launch {tool_config['name']}",
                                    variant="primary",
                                    elem_id=f"launch-{tool_id}"
                                )

                                stop_btn = gr.Button(
                                    "🛑 Stop",
                                    variant="stop",
                                    visible=False,
                                    elem_id=f"stop-{tool_id}"
                                )

                                # Web UI link
                                webui_link = gr.HTML(
                                    visible=False,
                                    elem_id=f"link-{tool_id}"
                                )

                        # Collapsible log viewer
                        with gr.Accordion(f"📋 Live Log - {tool_config['name']}", open=False):
                            log_output = gr.Textbox(
                                lines=15,
                                max_lines=20,
                                interactive=False,
                                show_copy_button=True,
                                elem_classes=["log-viewer"],
                                elem_id=f"log-{tool_id}"
                            )

                            with gr.Row():
                                refresh_log_btn = gr.Button("🔄 Refresh", size="sm")
                                clear_log_btn = gr.Button("🗑️ Clear", size="sm")

                        # Store components
                        tool_components[tool_id] = {
                            'status_html': tool_status_html,
                            'custom_args': custom_args,
                            'launch_btn': launch_btn,
                            'stop_btn': stop_btn,
                            'webui_link': webui_link,
                            'log_output': log_output,
                            'refresh_log_btn': refresh_log_btn,
                            'clear_log_btn': clear_log_btn
                        }

        # Store tool_components in demo for access in event handlers
        demo.tool_components = tool_components
        demo.hardware_profile = hardware_profile
        demo.process_states = process_states

        return demo

print("✅ Gradio interface structure created")


✅ Gradio interface structure created


In [6]:
# @title Main Launcher Cell - Section 6: Complete Interface with Events  { display-mode: "form" }
# @markdown ▶️ **Creates complete interface with all event handlers**

# ───────────────────────────────────────────────────────────────
# COMPLETE GRADIO INTERFACE WITH EVENT HANDLERS
# ───────────────────────────────────────────────────────────────

def create_complete_interface():
    """Create the complete Gradio interface with all event handlers"""

    with gr.Blocks(css=advanced_css, title="Unified WebUI Launcher", theme=gr.themes.Base()) as demo:

        # Global state management
        process_states = gr.State({})
        refresh_counter = gr.State(0)

        # Header section
        gr.HTML("""
        <div class="global-settings">
            <h1>🚀 Unified Multi-WebUI Stable Diffusion Launcher</h1>
            <p>Advanced multi-environment management with centralized assets and zero conflicts</p>
        </div>
        """)

        # System status dashboard
        with gr.Row():
            with gr.Column(scale=2):
                gr.Markdown("### 🌐 Global Configuration")

                with gr.Row():
                    workspace_dir = gr.Textbox(
                        label="Workspace Directory",
                        value=str(manager.workspace_dir),
                        interactive=False
                    )
                    models_dir = gr.Textbox(
                        label="Models Directory",
                        value=str(manager.models_dir),
                        interactive=False
                    )

                hardware_profile = gr.Dropdown(
                    label="Hardware Profile",
                    choices=[(profile['name'], key) for key, profile in manager.config['hardware_profiles'].items()],
                    value="medium_vram",
                    interactive=True
                )

            with gr.Column(scale=1):
                gr.Markdown("### 📊 System Status")

                status_summary = process_tracker.get_status_summary()

                with gr.Row():
                    total_tools_display = gr.HTML(
                        f'<div class="metric-card"><div class="metric-value">{status_summary["total"]}</div><div class="metric-label">Tools Available</div></div>'
                    )
                    running_count_display = gr.HTML(
                        f'<div class="metric-card"><div class="metric-value">{status_summary["running"]}</div><div class="metric-label">Currently Running</div></div>'
                    )

        # Tool management section
        gr.Markdown("## 🛠️ WebUI Tool Management")

        # Create tabs for each tool category
        with gr.Tabs():
            # Stable Diffusion tools tab
            with gr.TabItem("🎨 Stable Diffusion Tools"):
                sd_tools = {k: v for k, v in manager.config['tools'].items()
                           if v.get('category') == 'stable_diffusion'}

                tool_components = {}

                for tool_id, tool_config in sd_tools.items():
                    with gr.Group():
                        # Tool status display
                        tool_status_html = gr.HTML(
                            process_tracker.get_tool_status_html(tool_id),
                            elem_id=f"tool-status-{tool_id}"
                        )

                        with gr.Row():
                            with gr.Column(scale=3):
                                # Configuration options
                                custom_args = gr.Textbox(
                                    label="Custom Arguments",
                                    value=" ".join(tool_config.get('default_args', [])),
                                    placeholder="Additional launch arguments...",
                                    interactive=True
                                )

                            with gr.Column(scale=1):
                                # Control buttons
                                launch_btn = gr.Button(
                                    f"🚀 Launch {tool_config['name']}",
                                    variant="primary",
                                    elem_id=f"launch-{tool_id}"
                                )

                                stop_btn = gr.Button(
                                    "🛑 Stop",
                                    variant="stop",
                                    visible=False,
                                    elem_id=f"stop-{tool_id}"
                                )

                                # Web UI link
                                webui_link = gr.HTML(
                                    visible=False,
                                    elem_id=f"link-{tool_id}"
                                )

                        # Collapsible log viewer
                        with gr.Accordion(f"📋 Live Log - {tool_config['name']}", open=False):
                            log_output = gr.Textbox(
                                lines=15,
                                max_lines=20,
                                interactive=False,
                                show_copy_button=True,
                                elem_classes=["log-viewer"],
                                elem_id=f"log-{tool_id}"
                            )

                            with gr.Row():
                                refresh_log_btn = gr.Button("🔄 Refresh", size="sm")
                                clear_log_btn = gr.Button("🗑️ Clear", size="sm")

                        # Store components
                        tool_components[tool_id] = {
                            'status_html': tool_status_html,
                            'custom_args': custom_args,
                            'launch_btn': launch_btn,
                            'stop_btn': stop_btn,
                            'webui_link': webui_link,
                            'log_output': log_output,
                            'refresh_log_btn': refresh_log_btn,
                            'clear_log_btn': clear_log_btn
                        }

        # Batch operations section
        gr.Markdown("## ⚡ Batch Operations")

        with gr.Row():
            setup_all_btn = gr.Button(
                "🔧 Setup All Environments",
                variant="primary",
                size="lg"
            )

            stop_all_btn = gr.Button(
                "⏹️ Stop All Tools",
                variant="stop",
                size="lg"
            )

            refresh_status_btn = gr.Button(
                "🔄 Refresh Status",
                variant="secondary",
                size="lg"
            )

        # Status and logs area
        with gr.Row():
            status_output = gr.HTML(elem_id="global-status")

        # ═══════════════════════════════════════════════════════════
        # EVENT HANDLERS (INSIDE THE BLOCKS CONTEXT)
        # ═══════════════════════════════════════════════════════════

        # Event handler functions
        def launch_tool_handler(tool_id):
            """Create launch handler for specific tool"""
            def handler(custom_args_str, hardware_prof, proc_states):
                try:
                    # Parse custom arguments
                    custom_args = custom_args_str.split() if custom_args_str.strip() else []

                    # Launch the tool
                    success = manager.launch_tool(
                        tool_id,
                        custom_args=custom_args,
                        hardware_profile=hardware_prof
                    )

                    if success:
                        proc_states[tool_id] = 'starting'
                        tool_config = manager.config['tools'][tool_id]

                        return {
                            process_states: proc_states,
                            tool_components[tool_id]['launch_btn']: gr.update(visible=False),
                            tool_components[tool_id]['stop_btn']: gr.update(visible=True),
                            tool_components[tool_id]['webui_link']: gr.update(
                                value=f'<a href="http://localhost:{tool_config["port"]}" target="_blank" style="color: var(--primary-color);">🌐 Open {tool_config["name"]} WebUI</a>',
                                visible=True
                            ),
                            status_output: gr.update(
                                value=f'<div style="color: green;">✅ {tool_config["name"]} launched successfully</div>'
                            )
                        }
                    else:
                        tool_config = manager.config['tools'][tool_id]
                        return {
                            process_states: proc_states,
                            status_output: gr.update(
                                value=f'<div style="color: red;">❌ Failed to launch {tool_config["name"]}</div>'
                            )
                        }

                except Exception as e:
                    return {
                        process_states: proc_states,
                        status_output: gr.update(
                            value=f'<div style="color: red;">❌ Error launching {tool_id}: {str(e)}</div>'
                        )
                    }

            return handler

        def stop_tool_handler(tool_id):
            """Create stop handler for specific tool"""
            def handler(proc_states):
                try:
                    success = manager.stop_tool(tool_id)
                    tool_config = manager.config['tools'][tool_id]

                    if success:
                        proc_states[tool_id] = 'stopped'

                        return {
                            process_states: proc_states,
                            tool_components[tool_id]['launch_btn']: gr.update(visible=True),
                            tool_components[tool_id]['stop_btn']: gr.update(visible=False),
                            tool_components[tool_id]['webui_link']: gr.update(visible=False),
                            status_output: gr.update(
                                value=f'<div style="color: orange;">🛑 {tool_config["name"]} stopped</div>'
                            )
                        }
                    else:
                        return {
                            process_states: proc_states,
                            status_output: gr.update(
                                value=f'<div style="color: red;">❌ Failed to stop {tool_config["name"]}</div>'
                            )
                        }

                except Exception as e:
                    return {
                        process_states: proc_states,
                        status_output: gr.update(
                            value=f'<div style="color: red;">❌ Error stopping {tool_id}: {str(e)}</div>'
                        )
                    }

            return handler

        def refresh_logs_handler(tool_id):
            """Create log refresh handler"""
            def handler():
                logs = process_tracker.update_logs(tool_id)
                return gr.update(value=logs)
            return handler

        # Batch operation handlers
        def setup_all_handler():
            """Setup all environments"""
            try:
                results = manager.setup_all_tools()
                success_count = sum(1 for success in results.values() if success)
                total_count = len(results)

                status_html = f"<h4>Setup Results: {success_count}/{total_count} successful</h4><ul>"
                for tool_id, success in results.items():
                    tool_name = manager.config['tools'][tool_id]['name']
                    status = "✅" if success else "❌"
                    status_html += f'<li>{status} {tool_name}</li>'
                status_html += "</ul>"

                return gr.update(value=status_html)

            except Exception as e:
                return gr.update(value=f'<div style="color: red;">❌ Setup failed: {str(e)}</div>')

        def stop_all_handler(proc_states):
            """Stop all running tools"""
            try:
                stopped_count = 0
                for tool_id in manager.config['tools'].keys():
                    status = manager.get_process_status(tool_id)
                    if status.get('status') in ['starting', 'running']:
                        if manager.stop_tool(tool_id):
                            proc_states[tool_id] = 'stopped'
                            stopped_count += 1

                return {
                    process_states: proc_states,
                    status_output: gr.update(value=f'<div style="color: orange;">🛑 Stopped {stopped_count} tools</div>')
                }

            except Exception as e:
                return {
                    process_states: proc_states,
                    status_output: gr.update(value=f'<div style="color: red;">❌ Error stopping tools: {str(e)}</div>')
                }

        # Wire up event handlers for SD tools
        for tool_id in tool_components.keys():
            components = tool_components[tool_id]

            # Launch button
            components['launch_btn'].click(
                fn=launch_tool_handler(tool_id),
                inputs=[components['custom_args'], hardware_profile, process_states],
                outputs=[process_states, components['launch_btn'], components['stop_btn'],
                        components['webui_link'], status_output]
            )

            # Stop button
            components['stop_btn'].click(
                fn=stop_tool_handler(tool_id),
                inputs=[process_states],
                outputs=[process_states, components['launch_btn'], components['stop_btn'],
                        components['webui_link'], status_output]
            )

            # Refresh logs
            components['refresh_log_btn'].click(
                fn=refresh_logs_handler(tool_id),
                outputs=[components['log_output']]
            )

            # Clear logs
            components['clear_log_btn'].click(
                fn=lambda: gr.update(value=""),
                outputs=[components['log_output']]
            )

        # Wire up batch operations
        setup_all_btn.click(fn=setup_all_handler, outputs=[status_output])
        stop_all_btn.click(fn=stop_all_handler, inputs=[process_states], outputs=[process_states, status_output])

    return demo

print("✅ Complete interface function created")


✅ Complete interface function created


In [8]:
# @title Main Launcher Cell - Section 7: Launch Interface  { display-mode: "form" }
# @markdown ▶️ **Creates & launches the complete Gradio interface (no queue-kwargs error)**

# ───────────────────────────────────────────────────────────────
# 1)  Build the interface (returns a gr.Blocks object)
# ───────────────────────────────────────────────────────────────
print("🚀 Creating complete interface …")
demo = create_complete_interface()          # uses function from Section 6

# ───────────────────────────────────────────────────────────────
# 2)  Print some system info
# ───────────────────────────────────────────────────────────────
print_launch_info()                         # helper from Section 4

# ───────────────────────────────────────────────────────────────
# 3)  Launch – NO extra kwargs for queue()  (Gradio ≥4)
# ───────────────────────────────────────────────────────────────
try:
    print("⚡ Starting Gradio server …")
    (
        demo
        .queue()                            # just queue() – no concurrency_count / max_size
        .launch(
            server_name="0.0.0.0",
            server_port=7860,
            share=True,                     # public link
            show_error=True,
            quiet=True,
            inbrowser=False                 # don’t auto-open inside Colab
        )
    )
except Exception as e:
    print(f"❌ Error launching Gradio: {e}")
    raise

# ───────────────────────────────────────────────────────────────
# 4)  Quick-control widget for notebook users (optional)
# ───────────────────────────────────────────────────────────────
def create_quick_controls():
    import ipywidgets as widgets
    from IPython.display import display, clear_output

    tool_dropdown = widgets.Dropdown(
        options=[(cfg['name'], tid) for tid, cfg in manager.config['tools'].items()],
        description='WebUI:'
    )
    profile_dropdown = widgets.Dropdown(
        options=[(p['name'], key) for key, p in manager.config['hardware_profiles'].items()],
        value='medium_vram',
        description='Profile:'
    )
    setup_btn  = widgets.Button(description="🔧 Setup",  button_style='info')
    launch_btn = widgets.Button(description="🚀 Launch", button_style='success')
    stop_btn   = widgets.Button(description="🛑 Stop",   button_style='danger')
    out = widgets.Output()

    def _setup(_):
        with out:
            clear_output()
            tid = tool_dropdown.value
            print(f"Setting up {manager.config['tools'][tid]['name']} …")
            ok = manager.setup_tool(tid)
            print("✅ Done" if ok else "❌ Failed")

    def _launch(_):
        with out:
            clear_output()
            tid, prof = tool_dropdown.value, profile_dropdown.value
            print(f"Launching {manager.config['tools'][tid]['name']} ({prof}) …")
            ok = manager.launch_tool(tid, hardware_profile=prof)
            port = manager.config['tools'][tid]['port']
            print(f"✅ Running at http://localhost:{port}" if ok else "❌ Failed")

    def _stop(_):
        with out:
            clear_output()
            tid = tool_dropdown.value
            print(f"Stopping {manager.config['tools'][tid]['name']} …")
            ok = manager.stop_tool(tid)
            print("🛑 Stopped" if ok else "❌ Not running / error")

    setup_btn.on_click(_setup); launch_btn.on_click(_launch); stop_btn.on_click(_stop)
    display(widgets.VBox([tool_dropdown, profile_dropdown,
                          widgets.HBox([setup_btn, launch_btn, stop_btn]), out]))

# Display quick controls
create_quick_controls()

print("\n🎉 Unified launcher ready – use the GUI above or quick-control widget below.")


🚀 Creating complete interface …

🚀 UNIFIED WEBUI LAUNCHER STARTING
📁 Workspace: /workspace
🎨 Models: /data/models
🛠️ Tools: 2
💾 RAM: 11.5 GB available
💿 Disk: 69.8 GB available

🎯 Features:
   ✅ Multi-environment isolation (zero conflicts)
   ✅ Centralized asset management (storage optimization)
   ✅ Real-time process monitoring
   ✅ Advanced Gradio interface
   ✅ Batch operations support
   ✅ Hardware profile optimization

📋 Next Steps:
   1. Use 'Setup All Environments' to install WebUIs
   2. Place your models in the centralized directory
   3. Launch any WebUI with optimal settings
   4. Monitor processes and logs in real-time

⚡ Starting Gradio server …
* Running on public URL: https://0b344de448642e6ec0.gradio.live



🎉 Unified launcher ready – use the GUI above or quick-control widget below.
